# Problem Description.

This project is based in the section about Meta programming of the paper [1].
Before starting, we recommend you to read carefully that section, 
but you can skip the subsection on Guess-and-check.

We start with some preliminaries, and afterwards we describe the parts of this project.

[1] Kaminski, R., Romero, J., Schaub, T., & Wanko, P. (2020). How to build your own ASP-based system?! CoRR, abs/2008.06692.

## Preliminaries.

Let us consider the Latin Square problem:
given a quadratic board of size `s`, 
the goal is to fill each cell of the
board with some number from `1` to `s` such that 
no number occurs twice in the same row or column.

This problem can be represented by the following logic program:

In [1]:
%%file latin.lp
number(1..s).

1 { latin(X,Y,N) : number(N) } 1 :- number(X), number(Y).
1 { latin(X,Y,N) : number(Y) } 1 :- number(X), number(N).
1 { latin(X,Y,N) : number(X) } 1 :- number(Y), number(N).

#show latin/3.

Writing latin.lp


The paper [1] presents the following meta encoding `meta.lp`.

In [3]:
%%file meta.lp

conjunction(B) :- literal_tuple(B),
        hold(L) : literal_tuple(B, L), L > 0;
    not hold(L) : literal_tuple(B,-L), L > 0.

body(normal(B)) :- rule(_,normal(B)), conjunction(B).
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

  hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

#show.
#show T : output(T,B), conjunction(B).

Overwriting meta.lp


The file can be used to compute answer sets of a logic program `P`
given a reified version of the program `P` generated by `clingo`.
For example, the answer sets generated by this call:

In [5]:
!clingo latin.lp -c s=4 0 --quiet

clingo version 5.4.0
Reading from latin.lp
Solving...
SATISFIABLE

Models       : 576
Calls        : 1
Time         : 0.004s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.004s


are the same as those generated by this one:

In [7]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
!clingo reify.lp meta.lp 0 --quiet

clingo version 5.4.0
Reading from reify.lp ...
Solving...
SATISFIABLE

Models       : 576
Calls        : 1
Time         : 0.030s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.030s


You can check this equivalence using the following Python function.
It takes as input a string and some files that, together, specify a logic program.
The function returns a sorted version of the answer sets of the program.

In [8]:
import clingo

def run(program="", files=[]):
    ctl = clingo.Control(["0"])
    ctl.add("base", [], program)
    for f in files:
        ctl.load(f)
    ctl.ground([("base", [])])
    with ctl.solve(yield_=True) as handle:
        return sorted(sorted(m.symbols(shown=True) for m in handle))

Note that the answer sets returned consist only of atoms and terms that are shown.

We can check that the previous programs have the same answer sets as follows:

In [9]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
run("", ['reify.lp', 'meta.lp']) == run("#const s=4.", ['latin.lp'])

True

The comparison returns `True` because the answer sets of both program are the same.

## Part 1. From clingo logic programs to disjunctive logic programs

The task of this part is to write a meta encoding `meta-disjunctive.lp` that is similar to `meta.lp`,
but after grounding using `clingo` it generates a disjunctive logic program (without choice rules or weight rules).
The meta encoding may contain some aggregates of cardinality constraints in the bodies, 
but they should be completely evaluated by the grounder of `clingo`.

The new meta encoding `meta-disjunctive.lp` should behave like `meta.lp`, so that
it can be used to compute answer sets of a logic program `P`
given a reified version of the program `P` generated by `clingo`.
You can run the same test as before:

In [ ]:
!clingo latin.lp -c s=4 --output=reify > reify.lp
run("", ['reify.lp', 'meta-disjunctive.lp']) == run("", ['latin.lp'])

You can check if the meta encoding generates no ground choice rules or weight rules with this call:

In [ ]:
!clingo latin.lp              -c s=4 --output=reify > reify.lp
!clingo reify.lp meta-disjunctive.lp --output=reify

If the meta encoding is correct, 
then the output should contain no facts of predicate `weighted_literal_tuple/3`, 
and no facts of the form `rule(choice(t1),t2)` for any terms `t1` and `t2`.

### Translating choice rules

The meta-encoding `meta.lp` contains the rule
```
{ hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).
```
that generates the atoms of a choice rule whenever its body holds.
The first task of this part is to replace that rule by a set of normal rules.

In our course Answer Set Solving in Practice (https://teaching.potassco.org), 
in the Language part, we explain how to do this for propositional logic programs.
You can apply the same approach here.

### Translating weight rules

The meta-encoding `meta.lp` contains the rule
```
body(sum(B,G))  :- rule(_,sum(B,G)),
    #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
           W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.
```
that determines when the body of a weight rule holds, using a `#sum` aggregate in the body.
The second task of this part is to replace that rule by a set of normal rules.
These rules should not contain aggregates or cardinality constraints, 
unless they are completely evaluated by the grounder of `clingo`.

In our course Answer Set Solving in Practice (teaching.potassco.org),
in the Language part, 
we explain how to do this for cardinality rules occurring in propositional logic programs.
You can apply the same approach here, except that in this case, 
whenever a literal holds, instead of adding `1` to the accumulated value `k`, 
we have to add the corresponding weight `w` of the literal.

Note that the weights `w` generated by `clingo` in the reified output are always positive integers.
In other words, the third argument of the atoms of the predicate `weighted_literal_tuple/3` is always a positive integer.

It may be helpful to order the literals occurring in the weight rules.
You can do it with this rule:
```
weighted_literal_tuple(B,L,W,P+1) :- weighted_literal_tuple(B,L,W),
                                     P = #count{ LL,WW : weighted_literal_tuple(B,LL,WW), (LL,WW)<(L,W) }. 
```
Observe that it uses an aggregate, but since it only depends on `weighted_literal_tuple/3`, 
it can be compeltely evaluated by the grounder of `clingo`.


## Part 2. From tight normal programs to programs with choice rules and constraints.

In this part we restrict ourselves to normal programs with integrity constraints.
These programs do not contain disjunctions, choice rules or weight rules. 
Furthermore, we restrict ourselves to tight programs, i.e., 
programs `P` such that the positive atom dependency graph of `ground(P)` is acyclic. 

Note that these are the programs that we obtain from Part 1
if the original program is tight and non-disjunctive, 
as in the case of `latin.lp`.
To see this, please inspect the output of these calls:

In [ ]:
!clingo latin.lp              -c s=4 --output=reify > reify.lp
!clingo reify.lp meta-disjunctive.lp --output=reify

The task of this part is to write a meta encoding `meta-constraints.lp` that is similar to `meta.lp`,
but only consists of choices rules and constraints.
The new meta encoding `meta-constraints.lp` should behave like `meta.lp`, so that
it can be used to compute answer sets of a tight normal logic program `P` with integrity constraints,
given a reified version of the program `P` generated by `clingo`.
You can run a similar same test as before:

In [ ]:
!clingo latin.lp               -c s=4 --output=reify > reify1.lp
!clingo reify1.lp meta-disjunctive.lp --output=reify > reify2.lp
run("", ['reify2.lp', 'meta-constraints.lp']) == run("", ['latin.lp'])

In the course Answer Set Solving in Practice (teaching.potassco.org), 
in the Solving part, 
we show how to represent a tight logic program by a set of nogoods
such that the answer sets of the logic program correspond 
to the solutions of the nogoods.
For tight logic programs, those nogoods are called the completion nogoods.

You can use this result to translate tight normal logic programs `P` with integrity constraints,
to programs `P'`with choice rules and constraints.
The choice rules are used to generate all possible assignments, while
the constraints are used to represent the nogoods.
For example, the nogood `{Ta, Fb, Fc}` can be represented by the constraint `:- a, not b, not c.`.
In this way, the answer sets of `P'` correspond to the solutions of the set of completion nogoods, 
that, given the previous result from the course, correspond to the answer sets of `P`.

We already provide you with the choice rules and the show statements that you need:

In [11]:
%%file meta-constraints.lp

{conjunction(B)} :- literal_tuple(B).
{     hold(|L|)} :- literal_tuple(B,L).
{     hold(|A|)} :- atom_tuple(H,A).

#show.
#show T : output(T,B), conjunction(B).

Writing meta-constraints.lp


Your task is to add the constraints representing the nogoods.
You only need two constraints for the atom-oriented nogoods, and
three constraints for the body-oriented nogoods.